In [ ]:
import json
import random

import contextily as cx
import geojson
import matplotlib.pyplot as plt
import shapely
import trajgenpy.Logging
from matplotlib.lines import lineStyles
from numpy import poly
from trajgenpy import Utils
from trajgenpy.Geometries import (
    GeoMultiPolygon,
    GeoMultiTrajectory,
    GeoPolygon,
    GeoTrajectory,
    decompose_polygon,
    generate_sweep_pattern,
    get_sweep_offset,
)
from trajgenpy.Query import query_features

log = trajgenpy.Logging.get_logger()



Load the geojson environmental file

In [ ]:
polygon_file = "DemaScenarios/FlatTerrainNature.geojson"

# Load the GeoJSON data
with open(polygon_file, "r") as f:
    data = json.load(f)

# Extract the polygon coordinates
coordinates = data["features"][0]["geometry"]["coordinates"][0]

# Create the shapely Polygon object
p = GeoPolygon(shapely.Polygon(coordinates)).set_crs("EPSG:2197")

Extract the environmental feature with the specific tags

In [ ]:
# multi_traj.plot(color="red", linestyle="dashed")
# Load the routes.json dictionary

with open("transport.json", "r") as f:
    transports = json.load(f)
with open("tasks.json", "r") as f:
    tasks = json.load(f)
# Extract each key's list of "EPSG:2197" waypoints and plot them
colors = plt.cm.get_cmap("tab10", len(tasks))
for i, (key, tasks) in enumerate(tasks.items()):
    for task in tasks:
        trajectory = GeoTrajectory(
            shapely.geometry.LineString(task["_coords"]), crs="EPSG:2187"
        )
        trajectory.plot(add_points=False)
# for i, (key, tasks) in enumerate(tasks.items()):
#     for task in tasks:
#         trajectory = GeoTrajectory(
#             shapely.geometry.LineString(task["_coords"]), crs="EPSG:2187"
#         )
#         trajectory.plot(add_points=False, color=colors(i))

# for i, (key, agent_transports) in enumerate(transports.items()):
#     for transport in agent_transports:
#         trajectory = GeoTrajectory(
#             shapely.geometry.LineString(transport), crs="EPSG:2187"
#         )
#         trajectory.plot(
#             add_points=False, linestyle="dashed", alpha=0.5, color=colors(i)
#         )


# tags = {
#     "natural": ["water", "wetland"],
# }

# wetland = query_features(
#     GeoPolygon(shapely.Polygon(coordinates)),
#     {
#         "natural": ["water", "wetland"],
#     },
# )
# for f in wetland.values():
#     GeoMultiPolygon(f).set_crs("EPSG:2197").plot()

# roads = query_features(
#     GeoPolygon(shapely.Polygon(coordinates)),
#     {
#         "highway": ["service", "track"],
#     },
# )
# for road in roads.values():
#     GeoMultiTrajectory(road).set_crs("EPSG:2197").plot(color="red")

# p.buffer(10).plot(facecolor="none", edgecolor="black", linewidth=2)

Utils.plot_basemap(provider=cx.providers.CartoDB.Positron, crs="EPSG:2197")

# No axis on the plot
# plt.axis("equal")
plt.axis("off")
plt.tight_layout()
# Adjust the plot limits to fit the basemap
# plt.xlim(p.geometry.bounds[0], p.geometry.bounds[2])
# plt.ylim(p.geometry.bounds[1], p.geometry.bounds[3])
# Save the plot as an SVG file
plt.savefig("planned_routes.pdf", format="pdf")

plt.show()

In [ ]:
polygon = GeoPolygon(shapely.Polygon(coordinates)).set_crs("EPSG:2197").geometry

In [ ]:
from shapely import is_empty
import time
import shapely.plotting

shapely.plotting.plot_polygon(
    polygon, add_points=False, facecolor=(0, 0, 0, 0)
)
shrunken_polygon = polygon
shrunken_polygon : shapely.Polygon
polygon_list = []
def shrink_polygon(p: shapely.Polygon, buffer_size):
    if p.is_empty:
        return []
    shrunken_polygon = p.buffer(
        -buffer_size, join_style="bevel", single_sided=False
    )
    if shrunken_polygon.is_empty:
        return [p]

    return [p, *shrink_polygon(shrunken_polygon, buffer_size)]

def shrink_polygon_while(p: shapely.Polygon, buffer_size):
    polygon_list = []
    while not p.is_empty:
        polygon_list.append(p)
        p = p.buffer(-buffer_size, join_style="bevel", single_sided=False)
    return polygon_list

vertice_sum = 0
start_time = time.time()
# polygon_list = shrink_polygon_while(shrunken_polygon, 1)
polygon_list = shrink_polygon(shrunken_polygon, 20)
end_time = time.time()

print(f"Time taken to execute shrink_polygon: {end_time - start_time} seconds")

for poly_item in polygon_list:
    # if isinstance(poly_item, shapely.geometry.MultiPolygon):
    #     for p in poly_item.geoms:
    #         vertice_sum +=len(p.exterior.coords)
    # else:
    #     vertice_sum += len(poly_item.exterior.coords)
    shapely.plotting.plot_polygon(poly_item, add_points=False, facecolor=(0, 0, 0, 0))

plt.grid(False)
plt.show()

In [ ]:
import cv2
from collections import defaultdict

def mask_to_polygons(mask, epsilon=10.0, min_area=10.0):
    """Convert a mask ndarray (binarized image) to Multipolygons"""
    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE
    )
    if not contours:
        return shapely.MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = shapely.Polygon(
                shell=cnt[:, 0, :],
                holes=[
                    c[:, 0, :]
                    for c in cnt_children.get(idx, [])
                    if cv2.contourArea(c) >= min_area
                ],
            )
            all_polygons.append(poly)
    all_polygons = shapely.MultiPolygon(all_polygons)

    return all_polygons

In [ ]:
# TODO:
# * Extract features and load a probability map
# * Create a mask by using a fitting threshold value
